# 10. Labeling tool

*Made by David Serrano for his Final Bachelor Thesis*

This file takes the prediction of the system-wide tests of unseen sequences and displays 4 images of all the tracks to have the possibility of reviewing the track predicitons.

## 10.1. Environment Preparation

In [ ]:
# Useful general imports
from google.colab import drive

In [ ]:
# mount the google drive folder to access to all the files in there
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Add the folder called utils that has all the custom modules in .py files
import sys
sys.path.append('/content/drive/MyDrive/TFG/utils')

In [ ]:
from paths import PROJECT_ROOT, HDF5_FOLDER, JSON_FOLDER, USABLE_SEQUENCES, MAX_SQUARE_LEN

## 10.2. Analysis

In [ ]:
EXAMPLE_FILE = '20180920.034_AEQ11'

In [ ]:
# Load the sequnce example
import h5py
import time
from image_tools import computeDarkImage

print('Loading sequence file...')
f = h5py.File(HDF5_FOLDER + EXAMPLE_FILE + '.h5', 'r', driver='core')
t = time.time()
images = f['ROIP']['ROIP1']['ROIP1Data'][()]
timestamps = f['ROIP']['ROIP1']['ROIP1W7XTime'][()]
elapsed = time.time() - t
f.close()
print('Elapsed time reading: {}'.format(elapsed))

darkIm = computeDarkImage(images)

Loading sequence file...
Elapsed time reading: 90.69288158416748
Computing Dark image...
Elapsed time computing dark image: 3.9812443256378174


In [ ]:
# Load detections
import cv2
import numpy as np
from JSONLoader import loadData
from hdf5Loader import hdf5Loader
from dataPrep import sampleTrackDetections
from image_tools import preProcess, plotBBoxes, adjustGamma, showImage

# detections = loadData(PROJECT_ROOT + '/data/experiment/' + EXAMPLE_FILE + 'pred.h5.json')

detections = loadData(JSON_FOLDER + EXAMPLE_FILE + '.h5.json')

loader = hdf5Loader(HDF5_FOLDER + EXAMPLE_FILE + '.h5')

# number of tracks
maxTrackID = max(d[1] for d in detections)
trackDetections = []
print(EXAMPLE_FILE)

for trackID in range(maxTrackID + 1):
    trackDetections = []
    for det in detections:
        if det[1] == trackID:
            trackDetections.append(det)

    subsampledTrackDetections = sampleTrackDetections(trackDetections, n=4)

    print('TRACK ID: {} - label: {}'.format(trackID, subsampledTrackDetections[0][6]))

    images = []
    for n in range(len(subsampledTrackDetections)):
        image = loader.loadImage(subsampledTrackDetections[n][0]) - darkIm

        image = preProcess(image)

        image = adjustGamma(image)

        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

        image = cv2.putText(image, str(subsampledTrackDetections[n][0]),
                            (50, image.shape[0]-50), cv2.FONT_HERSHEY_SIMPLEX,
                            2, (0,0,255), 2, cv2.LINE_AA, False)
        
        image = plotBBoxes(image, [[subsampledTrackDetections[n][2],
                                   subsampledTrackDetections[n][3],
                                   subsampledTrackDetections[n][4],
                                   subsampledTrackDetections[n][5],
                                   subsampledTrackDetections[n][6]]])

        images.append(image)

    imageStacked1 = np.hstack((images[0], images[1]))
    imageStacked2 = np.hstack((images[2], images[3]))
    imageStacked = np.vstack((imageStacked1, imageStacked2))
    imageStacked = imageStacked = cv2.resize(imageStacked,
                                            (int(imageStacked.shape[1] / 3),
                                            int(imageStacked.shape[0] / 3)))

    showImage(imageStacked)

Output hidden; open in https://colab.research.google.com to view.